# Image Classification with AWS SageMaker and Keras with backend MXNet
<hr>
## Handle the dataset
1. Download the dataset
2. Uncompress it
3. Preprocess the data
4. Save the data x_train, y_train, x_test and y_test to pickle file

Folder Structure:

data/train: save x_train and y_train as pickled files with names train-x and train-y repsectively<br>
data/validation: save x_test and y_test as pickled files with names test-x and test-y repsectively

## Download Dataset

In [ ]:
!wget https://www.cs.utoronto.ca/~kriz/cifar-10-python.tar.gz
!tar xzvf cifar-10-python.tar.gz
!rm -rf cifar-10-python.tar.gz

In [ ]:
import pickle
import numpy as np
from os import listdir
from os.path import isfile, join
import os

# Function to unpickle the dataset
def unpickle_all_data(directory):
    
    # Initialize the variables
    train = dict()
    test = dict()
    train_x = []
    train_y = []
    test_x = []
    test_y = []
    
    # Iterate through all files that we want, train and test
    # Train is separated into batches
    for filename in listdir(directory):
        if isfile(join(directory, filename)):
            
            # The train data
            if 'data_batch' in filename:
                print('Handing file: %s' % filename)
                
                # Opent the file
                with open(directory + '/' + filename, 'rb') as fo:
                    data = pickle.load(fo, encoding='bytes')

                if 'data' not in train:
                    train['data'] = data[b'data']
                    train['labels'] = np.array(data[b'labels'])
                else:
                    train['data'] = np.concatenate((train['data'], data[b'data']))
                    train['labels'] = np.concatenate((train['labels'], data[b'labels']))
            # The test data
            elif 'test_batch' in filename:
                print('Handing file: %s' % filename)
                
                # Open the file
                with open(directory + '/' + filename, 'rb') as fo:
                    data = pickle.load(fo, encoding='bytes')
                
                test['data'] = data[b'data']
                test['labels'] = data[b'labels']
    
    # Manipulate the data to the propper format
    for image in train['data']:
        train_x.append(np.transpose(np.reshape(image,(3, 32,32)), (1,2,0)))
    train_y = [label for label in train['labels']]
    
    for image in test['data']:
        test_x.append(np.transpose(np.reshape(image,(3, 32,32)), (1,2,0)))
    test_y = [label for label in test['labels']]
    
    # Transform the data to np array format
    train_x = np.array(train_x)
    train_y = np.array(train_y)
    test_x = np.array(test_x)
    test_y = np.array(test_y)
    
    return (train_x, train_y), (test_x, test_y)

(x_train, y_train), (x_test, y_test) = unpickle_all_data(os.getcwd() + '/cifar-10-batches-py/')

with open('data/validation/test-x', 'wb') as handle:
    pickle.dump(x_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('data/validation/test-y', 'wb') as handle:
    pickle.dump(y_test, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('data/train/train-x', 'wb') as handle:
    pickle.dump(x_train, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('data/train/train-y', 'wb') as handle:
    pickle.dump(y_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Start a SageMaker session

In [ ]:
import sagemaker
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

## Validate your configuration

1. DockerFile for your custom container that holds your code
2. keras.json configuration
3. python file that defined your model

In [ ]:
!cat Dockerfile.gpu

In [ ]:
!cat keras.jsonas.json

In [ ]:
!tail cifar_10_keras.py

## Copy the Dockerfile, keras.json and cifar_10_keras.py into build folder

In [ ]:
# Copy Dockerfiles
!cp Dockerfile.* build/

In [ ]:
# Copy training script and config file
!cp cifar_10_keras.py build/
!cp keras.json build/

## GPU Settings for training your algorithm

In [ ]:
repo_name = 'keras-mxnet-gpu' # ECR repository
image_tag = 'keras-mxnet1.2.0-gpu-py3' # ECR image tag
base_job_name = 'keras-mxnet-cifar-10-cnn' # SageMaker training prefix

%env dockerfile Dockerfile.gpu

train_instance_type='ml.p3.2xlarge'
gpu_count=1
batch_size=64

## Common settings

In [ ]:
%env account {account}
%env region {region}
%env repo_name {repo_name}
%env image_tag {image_tag}

## Create Repository and login

Before advancing to this step make sure you have an ECR up and running with the same name as in the vaiable `repo_name`

If you encounter an authentication error, make sure you have given the appropriate access to the repository

In [ ]:
%%sh

aws ecr describe-repositories --repository-names $repo_name > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name $repo_name > /dev/null
fi

$(aws ecr get-login --region $region --no-include-email)

## Build the Docker image

In [ ]:
%cd build
!docker build -t $image_tag -f $dockerfile .
%cd ..

In [ ]:
# Assign a tag to the Docker image
!docker tag $image_tag $account.dkr.ecr.$region.amazonaws.com/$repo_name:latest

In [ ]:
# Make sure you see your image with a tag `:latest` at the end of the name
!docker images -a

## Push docker image to ECR

In [ ]:
!docker push $account.dkr.ecr.$region.amazonaws.com/$repo_name:latest

## Upload dataset to S3

In [ ]:
local_directory = 'data'
prefix          = repo_name+'/input'

# Upload dataset to S3 and set the train and validation path
train_input_path      = sess.upload_data(local_directory+'/train/',      key_prefix=prefix+'/train')
validation_input_path = sess.upload_data(local_directory+'/validation/', key_prefix=prefix+'/validation')

## Create an estimator and run your image

In [ ]:
output_path = 's3://{}/{}/output'.format(sess.default_bucket(), repo_name)
image_name  = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, repo_name)

print(output_path)
print(image_name)

estimator = sagemaker.estimator.Estimator(
                       image_name=image_name,
                       base_job_name=base_job_name,
                       role=role, 
                       train_instance_count=1, 
                       train_instance_type=train_instance_type,
                       output_path=output_path,
                       sagemaker_session=sess)

estimator.set_hyperparameters(lr=0.0001, epochs=10, gpus=gpu_count, batch_size=batch_size)

## Train the algorithm

In [ ]:
estimator.fit({'training': train_input_path, 'validation': validation_input_path})